In [2]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
import datetime

In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [3]:
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [4]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [5]:
race = pd.read_csv('C:/Users/vigrose/PhysicianRaceEthnicity_2_2022-03-01.csv')
race.medical_education_number = use.fix_me(race.medical_education_number)

In [6]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [7]:
ME_QUERY = \
    f"""
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME,
    P.UPDATE_TMSTP,
    P.ACTIVE_IND
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [8]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 9
    """
entity_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [9]:
NAME_QUERY = \
    """
    SELECT
    FIRST_NM, MIDDLE_NM, LAST_NM, SUFFIX, LABEL_NM, NM_TYPE, PARTY_ID
    FROM
    AMAEDW.PERSON_NM
    WHERE
    THRU_DT IS NULL
    AND
    NM_TYPE='LN'
    """
NAMES = pd.read_sql(con=AMAEDW, sql=NAME_QUERY)

In [10]:
PERSON_QUERY = \
    """
    SELECT
    BIRTH_DT, BIRTH_CITY_NM, BIRTH_STATE_ID, BIRTH_COUNTRY_ID, GENDER_CD, MORTALITY_STS_CD, DEATH_DT, MED_EDU_NBR, PARTY_ID
    FROM
    AMAEDW.PERSON
    WHERE
    ACTIVE_IND = 'Y'
    """
PERSONS = pd.read_sql(con=AMAEDW, sql=PERSON_QUERY)

In [11]:
IDS = pd.merge(entity_ids, party_ids, on='PARTY_ID')
test = pd.merge(IDS, ov_me, left_on='ME', right_on='medical_education_number', how='right')
test_cont = pd.merge(test, NAMES, on='PARTY_ID',how='left').drop_duplicates()
test_cont = pd.merge(test_cont, race, on='medical_education_number', how='left')
ALL = pd.merge(test_cont, PERSONS, on='PARTY_ID',how='left').drop_duplicates()

In [12]:
today = datetime.date.today()
types = ['Physician','Student','Resident']
dead_mes = list(ALL[(ALL.MORTALITY_STS_CD=='C')].ME)
all_mes = list(ALL.ME)
us_mes = list(ALL[(ALL.BIRTH_COUNTRY_ID==6705)].ME)
elements = [
    {'data_element': 'Party ID',
     'element':'PARTY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Entity ID',
     'element':'ENTITY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'First Name',
     'element':'FIRST_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Middle Name',
     'element':'MIDDLE_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Last Name',
     'element':'LAST_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Name Suffix',
     'element':'SUFFIX',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Label Name',
     'element':'LABEL_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Name Type',
     'element':'NM_TYPE',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Person Type',
     'element':'type',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Race/Ethnicity',
     'element':'race_ethnicity',
    'null': 'Unknown',
    'universe': all_mes},
    {'data_element': 'Birth Date',
     'element':'BIRTH_DT',
    'null': datetime.date(1900, 1, 1),
    'universe':all_mes},
    {'data_element': 'Birth City',
     'element':'BIRTH_CITY_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Birth State',
     'element':'BIRTH_STATE_ID',
    'null': '',
    'universe': us_mes},
    {'data_element': 'Birth Country',
     'element':'BIRTH_COUNTRY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Gender',
     'element':'GENDER_CD',
    'null': '',
    'universe':all_mes},
    {'data_element': 'Death Date',
     'element':'DEATH_DT',
    'null': '',
    'universe':dead_mes},     
]
dict_list = []
for typo in types:
    total = len(ALL[ALL.type==typo])
    for element in elements:
        universe = len(ALL[(ALL.type==typo)&(ALL.ME.isin(element['universe']))])
        complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        true_complete = len(ALL[(ALL.type==typo)&(ALL[element['element']]!=element['null'])&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness'
        }
        dict_list.append(new_dict)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [13]:
pd.DataFrame(dict_list).to_csv(f'../../Data/Measurement/Person_Completeness_{str(datetime.date.today())}.csv', index=False)

In [1]:
elements

NameError: name 'elements' is not defined

In [6]:
elements = [
    {'data_element': 'Party ID',
     'element':'PARTY_ID',
    'null': ['']},
    {'data_element': 'Entity ID',
     'element':'ENTITY_ID',
    'null': ['']},
    {'data_element': 'First Name',
     'element':'FIRST_NM',
    'null': ['']},
    {'data_element': 'Middle Name',
     'element':'MIDDLE_NM',
    'null': ['']},
    {'data_element': 'Race/Ethnicity',
     'element':'race_ethnicity',
    'null': ['Unknown']},
    {'data_element': 'Birth Date',
     'element':'BIRTH_DT',
    'null': [datetime.date(1900, 1, 1)]} 
]

In [8]:
pd.DataFrame(elements).to_csv('../../Data/Measurement/Test.csv', index=False)

In [10]:
country = pd.read_csv('../../Data/Measurement/COUNTRY.csv')

In [13]:
country.COUNTRY_ID.isin(ALL.BIRTH_COUNTRY_ID)

NameError: name 'ALL' is not defined

In [14]:
ALL

NameError: name 'ALL' is not defined